In [15]:
# Import necessary libraries
import pandas as pd

# File paths for the uploaded datasets
customers_file = 'customers.csv'
products_file = 'Products.csv'
transactions_file = 'Transactions.csv'

# Load datasets
customers = pd.read_csv(customers_file)
products = pd.read_csv(products_file)
transactions = pd.read_csv(transactions_file)

In [22]:
customers['SignupDate'] = pd.to_datetime(customers['SignupDate'], errors='coerce')
transactions['TransactionDate'] = pd.to_datetime(transactions['TransactionDate'], errors='coerce')


In [32]:

# Recalculate TotalValue where it doesn't match Price * Quantity
transactions['TotalValue'] = transactions.apply(
    lambda row: row['Price'] * row['Quantity'] if row['TotalValue'] != row['Price'] * row['Quantity'] else row['TotalValue'],
    axis=1
)

# merge datasets after removing integrity issues
clean_transactions_products = transactions.merge(products, on='ProductID', how='left')
clean_merged_data = clean_transactions_products.merge(customers, on='CustomerID', how='left')

# Check the first few rows of the cleaned merged dataset
clean_merged_data.head(5)

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,ProductName,Category,Price_y,CustomerName,Region,SignupDate
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Andrea Jenkins,Europe,2022-12-03
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Brittany Harvey,Asia,2024-09-04
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Kathryn Stevens,Europe,2024-04-04
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Travis Campbell,South America,2024-04-11
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Timothy Perez,Europe,2022-03-15


In [33]:

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# Filter the first 20 customers
top_20_customers = customers[customers["CustomerID"].isin([f"C{str(i).zfill(4)}" for i in range(1, 21)])]



# Feature engineering: Aggregating transaction data for each customer
customer_features = (
    clean_merged_data.groupby("CustomerID")
    .agg(
        total_spent=("TotalValue", "sum"),
        total_quantity=("Quantity", "sum"),
        num_transactions=("TransactionID", "count"),
        avg_transaction_value=("TotalValue", "mean"),
    )
    .reset_index()
)

customer_features


,CustomerID,total_spent,total_quantity,num_transactions,avg_transaction_value
0,C0001,3354.52,12,5,670.904000
1,C0002,1862.74,10,4,465.685000
2,C0003,2725.38,14,4,681.345000
3,C0004,5354.88,23,8,669.360000
4,C0005,2034.24,7,3,678.080000
...,...,...,...,...,...
194,C0196,4982.88,12,4,1245.720000
195,C0197,1928.65,9,3,642.883333
196,C0198,931.83,3,2,465.915000
197,C0199,1979.28,9,4,494.820000


In [34]:
# Adding region and category preferences as one-hot encoded features
region_dummies = pd.get_dummies(clean_merged_data[["CustomerID", "Region"]].drop_duplicates(), columns=["Region"])
category_prefs = pd.get_dummies(clean_merged_data[["CustomerID", "Category"]], columns=["Category"]).groupby("CustomerID").sum()

# Combine all features
customer_features = customer_features.merge(region_dummies, on="CustomerID").merge(category_prefs, on="CustomerID")

# Normalize the features for cosine similarity
scaler = StandardScaler()
features_scaled = scaler.fit_transform(customer_features.drop("CustomerID", axis=1))

# Calculate cosine similarity
similarity_matrix = cosine_similarity(features_scaled)



In [35]:
# Find top 3 similar customers for each of the first 20 customers
lookalike_results = {}
for idx, customer_id in enumerate(top_20_customers["CustomerID"]):
    similarities = similarity_matrix[idx]
    top_3_indices = similarities.argsort()[-4:-1][::-1]  # Top 3 excluding self
    top_3_customers = [(customer_features.iloc[i]["CustomerID"], similarities[i]) for i in top_3_indices]
    lookalike_results[customer_id] = top_3_customers

# Save results to Lookalike.csv
lookalike_df = pd.DataFrame(
    [
        {"CustomerID": k, "Lookalikes": str(v)}
        for k, v in lookalike_results.items()
    ]
)
lookalike_path = "Lookalike.csv"
lookalike_df.to_csv(lookalike_path, index=False)

lookalike_path


'Lookalike.csv'